In [1]:
import pandas as pd

In [2]:
import nltk
import numpy as np
import re
import string

import seaborn as sns
import matplotlib.pyplot as plt
from nltk.chunk import ne_chunk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from collections import Counter
import warnings
warnings.filterwarnings("ignore")

In [46]:
import spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.8 MB/s eta 0:00:0000:010:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
import multiprocessing
from pandarallel import pandarallel
import spacy
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
!pip install vaderSentiment

In [5]:
from google.cloud import storage
storage_client = storage.Client()

bucket_name = "nlp_finalproject"

bucket = storage_client.bucket(bucket_name)
print(f"Bucket {bucket.name} connected.")

Bucket nlp_finalproject connected.


In [26]:
df_final_sent = pd.read_parquet("gs://nlp_finalproject/NER_Clean.parquet", engine='pyarrow')
df_final_sent.head(5)

,date,title,text,year,cleaned,clean_title,ORG_Ent,Person_Ent
0,2020-02-26,LegalTech Artificial Intelligence Market 2019 ...,LegalTech Artificial Intelligence Market 2019 ...,2020,LegalTech Artificial Intelligence Market 2019 ...,LegalTech Artificial Intelligence Market 2019 ...,[LegalTech Artificial Intelligence Market Tec...,[]
4,2020-08-07,Two dead as AI Express flight skids off Kozhik...,Two dead as AI Express flight skids off Kozhik...,2020,Two dead as AI Express flight skids off Kozhik...,Two dead as AI Express flight skids off Kozhik...,"[AI Express, PostBeyond BygoneEpic PowerIn Ret...",[]
7,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...,2021,MulticoreWare Inc. Becomes CEVA s Trusted Part...,MulticoreWare Inc. Becomes CEVA s Trusted Part...,"[MulticoreWare Inc., Trusted Partner for Imagi...",[]
8,2022-10-06,Healthcare Artificial Intelligence Market Anal...,\n\nHealthcare Artificial Intelligence Market ...,2022,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,[Healthcare Artificial Intelligence Market Ana...,[]
9,2020-04-29,Artificial Intelligence As A Service Market : ...,\n\nArtificial Intelligence As A Service Marke...,2020,Artificial Intelligence As A Service Market : ...,Artificial Intelligence As A Service Market : ...,"[Artificial Intelligence As A Service Market, ...",[]


In [7]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [8]:
num_processors = multiprocessing.cpu_count()
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False, progress_bar = True)

INFO: Pandarallel will run on 15 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [31]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [30]:
def analyze_sentiment(text):
    sentiment_scores = sid.polarity_scores(text)
    return sentiment_scores['compound']

df_final_sent['sentiment_score'] = df_final_sent['cleaned'].parallel_apply(analyze_sentiment)

In [65]:
df_final_sent['sentiment'] = df_final_sent['sentiment_score'].apply(lambda score: 'Positive' if score > 0.95 else ('Negative' if score < -0.90 else 'Neutral'))

In [66]:
df_final_sent['sentiment'].value_counts()

sentiment
Positive    145447
Neutral      11433
Negative      6441
Name: count, dtype: int64

In [55]:
# Displaying the data with sentiment labels
df_final_sent[['title','cleaned', 'sentiment','sentiment_score']]

,title,cleaned,sentiment,sentiment_score
0,LegalTech Artificial Intelligence Market 2019 ...,LegalTech Artificial Intelligence Market 2019 ...,Positive,0.9991
4,Two dead as AI Express flight skids off Kozhik...,Two dead as AI Express flight skids off Kozhik...,Negative,-0.9728
7,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,MulticoreWare Inc. Becomes CEVA s Trusted Part...,Positive,0.9981
8,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,Positive,0.9993
9,Artificial Intelligence As A Service Market : ...,Artificial Intelligence As A Service Market : ...,Positive,0.9996
...,...,...,...,...
200429,Generative AI filled us with wonder in 2023 - ...,Generative AI filled us with wonder in 2023 - ...,Positive,1.0000
200430,Google Cloud expands developer tools and data ...,Google Cloud expands developer tools and data ...,Positive,0.9999
200431,Twitter working on AI despite Musk call for gl...,Twitter working on AI despite Musk call for gl...,Positive,0.9980
200433,OpenAI Restores Access To ChatGPT In Italy Aft...,OpenAI Restores Access To ChatGPT In Italy Aft...,Positive,0.8867


In [45]:
df_final_sent = df_final_sent.drop(['Person_Ent'],axis=1)

In [47]:
df_final_sent.head(5)

,date,title,text,year,cleaned,clean_title,ORG_Ent,sentiment,sentiment_score
0,2020-02-26,LegalTech Artificial Intelligence Market 2019 ...,LegalTech Artificial Intelligence Market 2019 ...,2020,LegalTech Artificial Intelligence Market 2019 ...,LegalTech Artificial Intelligence Market 2019 ...,[LegalTech Artificial Intelligence Market Tec...,Positive,0.9991
4,2020-08-07,Two dead as AI Express flight skids off Kozhik...,Two dead as AI Express flight skids off Kozhik...,2020,Two dead as AI Express flight skids off Kozhik...,Two dead as AI Express flight skids off Kozhik...,"[AI Express, PostBeyond BygoneEpic PowerIn Ret...",Negative,-0.9728
7,2021-02-25,MulticoreWare Inc. Becomes CEVA’s Trusted Part...,\n\nMulticoreWare Inc. Becomes CEVA’s Trusted ...,2021,MulticoreWare Inc. Becomes CEVA s Trusted Part...,MulticoreWare Inc. Becomes CEVA s Trusted Part...,"[MulticoreWare Inc., Trusted Partner for Imagi...",Positive,0.9981
8,2022-10-06,Healthcare Artificial Intelligence Market Anal...,\n\nHealthcare Artificial Intelligence Market ...,2022,Healthcare Artificial Intelligence Market Anal...,Healthcare Artificial Intelligence Market Anal...,[Healthcare Artificial Intelligence Market Ana...,Positive,0.9993
9,2020-04-29,Artificial Intelligence As A Service Market : ...,\n\nArtificial Intelligence As A Service Marke...,2020,Artificial Intelligence As A Service Market : ...,Artificial Intelligence As A Service Market : ...,"[Artificial Intelligence As A Service Market, ...",Positive,0.9996


In [48]:
nlp = spacy.load("en_core_web_sm")

In [50]:
%%time
def extract_entities(text):
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ == "PERSON"]
    return entities

df_final_sent['Person_Ent'] = df_final_sent['cleaned'].parallel_apply(extract_entities)

CPU times: user 3min, sys: 20.7 s, total: 3min 21s
Wall time: 1h 5min 18s


In [57]:
df_final_sent[['Person_Ent','year']]

,Person_Ent,year
0,"[Aristocrat Leisure, Thermo Fisher, Woodley Eq...",2020
4,"[Kozhikode airportTop Toggle, Safari, safari,...",2020
7,"[Erez Natan, Soumendra Mohanty]",2021
8,"[AiCure, Web , Profile Follow]",2022
9,"[Emerging Trends, IRIS AI, Get Sample PDF, Pur...",2020
...,...,...
200429,"[Which Windows, Ray-Ban, Google Microsoft, Ran...",2023
200430,"[Google Cloud, ZDNET ...",2023
200431,"[Musk, Ghana , reCAPTC...",2023
200433,"[Ban, Zee News, Garante, Astro, OpenAI, Garant...",2023


In [56]:
tmp = df_final_sent[['Person_Ent','year']]
plot_tmp = tmp.explode('Person_Ent')
Person_counts = plot_tmp['Person_Ent'].value_counts()

In [76]:
Person_counts.head(30)

Person_Ent
Twitter               30674
Sam Altman            18859
Biden                 15772
GPT-4                 15758
Elon Musk             11234
Musk                   9921
WhatsApp               9488
Size                   6650
AI                     6547
AdvertisingAt Gray     6154
Generative AI          5902
Joe Biden              5780
Inquiries Request      5554
Telecomm               5347
OpenAI                 5034
LinkedIn Email         4968
Google Cloud           4254
Anthropic              4126
Midjourney             4058
Math                   4037
COVID-19               3964
Satya Nadella          3962
Forgot                 3864
Don                    3855
Center ProfNet         3611
Machine Learning       3604
Dashboard              3556
Bloomberg              3543
Facebook Twitter       3497
Donald Trump           3415
Name: count, dtype: int64

In [75]:
df_final_sent.to_parquet('gs://nlp_finalproject/SentimentAnalysis_Clean.parquet')